In [4]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(12,10)})

# import sys
# !conda list Check the packages installed

In [5]:
# Loading the cumulative raw data

raw_data_confirmed = pd.read_csv('E:\\Prasad\\Tableau_Files\\Python-Tutorials-master\\CoronaVirus\\time_series_19-covid-Confirmed.csv')
raw_data_deaths = pd.read_csv('E:\\Prasad\\Tableau_Files\\Python-Tutorials-master\\CoronaVirus\\time_series_19-covid-Deaths.csv')
raw_data_Recovered = pd.read_csv('E:\\Prasad\Tableau_Files\\Python-Tutorials-master\\CoronaVirus\\time_series_19-covid-Recovered.csv')

print("The Shape of Cornirmed is: ", raw_data_confirmed.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered.shape)

raw_data_confirmed.head()

The Shape of Cornirmed is:  (225, 50)
The Shape of Cornirmed is:  (225, 50)
The Shape of Cornirmed is:  (225, 50)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,990,990,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,410,410,411,413,414,414,418,418,422,426
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,296,296,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,102,119,120


In [6]:
raw_data_Recovered[raw_data_Recovered['Country/Region'] == 'US']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20
35,"King County, WA",US,47.6062,-122.3321,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
36,"Cook County, IL",US,41.7377,-87.6976,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
46,"Los Angeles, CA",US,34.0522,-118.2437,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,"San Benito, CA",US,36.5761,-120.9876,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66,"Madison, WI",US,43.0731,-89.4012,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
68,"San Diego County, CA",US,32.7157,-117.1611,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
69,"San Antonio, TX",US,29.4241,-98.4936,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,"Omaha, NE (From Diamond Princess)",US,41.2545,-95.9758,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,"Travis, CA (From Diamond Princess)",US,38.2721,-121.9399,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75,"Lackland, TX (From Diamond Princess)",US,29.3829,-98.6134,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Un-Pivoting the data

raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_Recovered2 = pd.melt(raw_data_Recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])


print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered2.shape)


raw_data_confirmed2.head()

The Shape of Cornirmed is:  (10350, 6)
The Shape of Cornirmed is:  (10350, 6)
The Shape of Cornirmed is:  (10350, 6)


,Province/State,Country/Region,Lat,Long,Date,value
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0


In [9]:
# Converting the new column to dates

raw_data_confirmed2['Date'] = pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date'] = pd.to_datetime(raw_data_deaths2['Date'])
raw_data_Recovered2['Date'] = pd.to_datetime(raw_data_Recovered2['Date'])


In [10]:
# Renaming the Values
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_Recovered2.columns = raw_data_Recovered2.columns.str.replace('value', 'Recovered')

In [11]:
# Investigating the NULL values
raw_data_Recovered2.isnull().sum()

Province/State    4324
Country/Region       0
Lat                  0
Long                 0
Date                 0
Recovered            0
dtype: int64

In [12]:
# Dealing with NULL values

raw_data_confirmed2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_deaths2['Province/State'].fillna(raw_data_deaths2['Country/Region'], inplace=True)
raw_data_Recovered2['Province/State'].fillna(raw_data_Recovered2['Country/Region'], inplace=True)

raw_data_confirmed2.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
dtype: int64

In [13]:
# printing shapes before the join
print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered2.shape)

The Shape of Cornirmed is:  (10350, 6)
The Shape of Cornirmed is:  (10350, 6)
The Shape of Cornirmed is:  (10350, 6)


In [14]:
raw_data_confirmed2.isnull().sum()
raw_data_deaths2.isnull().sum()
raw_data_Recovered2.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Recovered         0
dtype: int64

In [15]:
# Full Joins

# Confirmed with Deaths
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

# full join with Recovered
full_join = full_join.merge(raw_data_Recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.head()

Shape of first join:  (10350, 7)
Shape of second join:  (10350, 8)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0


In [16]:
# checking for null values (especially long and lat)
full_join.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
Deaths            0
Recovered         0
dtype: int64

In [17]:
# Adding Month and Year as a new Column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [18]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020


In [19]:
#creating a new df    
test = full_join[full_join['Province/State'] == 'Anhui']

#creating a new df    
full_join2 = test.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = test.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'outer',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']


test.head()
full_join2.head()
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN
1,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020,1.0,0.0,0.0,2020-01-23,2020-01-22,8.0
2,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020,9.0,0.0,0.0,2020-01-24,2020-01-23,6.0
3,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020,15.0,0.0,0.0,2020-01-25,2020-01-24,24.0
4,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020,39.0,0.0,0.0,2020-01-26,2020-01-25,21.0


In [20]:
test.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020
225,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9,0,0,Jan-2020
450,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15,0,0,Jan-2020
675,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39,0,0,Jan-2020
900,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60,0,0,Jan-2020


In [21]:
## Applying it on all dataset

#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

full_join3.head()

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(10350, 17)


In [22]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [23]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

C:\Users\Prasad\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
# Creating additional slicer for easy of use

full_join3['Hubei Vs Rest of the World'] = 'Rest of the World'
full_join3['Hubei Vs Rest of the World'].loc[full_join3['Province/State'] == 'Hubei'] = 'Hubei - Virus birth'

#full_join3[full_join3['Province/State'] == 'Hubei']

In [25]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily,Hubei Vs Rest of the World
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0,Rest of the World
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,14.0,0.0,0.0,Rest of the World
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,6.0,0.0,0.0,Rest of the World
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0,Rest of the World
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,0.0,0.0,0.0,Rest of the World


In [26]:
# Exporting the data

# Setting my path
path = "E:\\Thesis\\Final_CSV"

# Changing my CWD
os.chdir(path)

full_join3.to_csv('CoronaVirus PowerBI Raw', sep='\t')

In [27]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily,Hubei Vs Rest of the World
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0,Rest of the World
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,14.0,0.0,0.0,Rest of the World
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,6.0,0.0,0.0,Rest of the World
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0,Rest of the World
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,0.0,0.0,0.0,Rest of the World


In [28]:
# puting unique values in list
dates = full_join3['Date'].unique()

#creating a df with unique
dates = pd.DataFrame(dates, columns=['Date'])

# Ordering df
dates = dates.sort_values(by=['Date'])

# Creating an ordered list now
dates = full_join3['Date'].unique()

print(full_join3.shape)

try:
    del concat_data
except:
    print()
    
try:
    del final_concat_data
except:
    print()
    

for i in dates:
    new_data = full_join3[full_join3['Date'] == i]
    new_data['Cumulative Date'] = i
    print(i)
    
    try:     
        concat_data = pd.concat([concat_data, new_data], ignore_index = True)
        concat_data['Cumulative Date 2'] = i
        print(concat_data['Date'].unique())
        
        try:
            final_concat_data = pd.concat([final_concat_data, concat_data], ignore_index = True)
        except:
            final_concat_data = concat_data

    except:
        concat_data = new_data
        
print(final_concat_data.shape)

(10350, 13)


2020-01-22T00:00:00.000000000
2020-01-23T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000']


C:\Users\Prasad\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Prasad\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\Prasad\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


2020-01-24T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000']
2020-01-25T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000']
2020-01-26T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000']
2020-01-27T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000']
2020-01-28T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000']
2020-01-29T00:00:00.000000000
[

In [29]:
# Exporting the data

final_concat_data.to_csv('CoronaVirus PowerBI Raw - Cumulative Test', sep='\t')

In [30]:
final_concat_data.head()

,Confirmed,Confirmed Daily,Country/Region,Cumulative Date,Cumulative Date 2,Date,Deaths,Deaths Daily,Hubei Vs Rest of the World,Lat,Long,Month-Year,Province/State,Recovered,Recovered Daily
0,1,1.0,Mainland China,2020-01-22,2020-01-23,2020-01-22,0,0.0,Rest of the World,31.8257,117.2264,Jan-2020,Anhui,0,0.0
1,14,14.0,Mainland China,2020-01-22,2020-01-23,2020-01-22,0,0.0,Rest of the World,40.1824,116.4142,Jan-2020,Beijing,0,0.0
2,6,6.0,Mainland China,2020-01-22,2020-01-23,2020-01-22,0,0.0,Rest of the World,30.0572,107.8740,Jan-2020,Chongqing,0,0.0
3,1,1.0,Mainland China,2020-01-22,2020-01-23,2020-01-22,0,0.0,Rest of the World,26.0789,117.9874,Jan-2020,Fujian,0,0.0
4,0,0.0,Mainland China,2020-01-22,2020-01-23,2020-01-22,0,0.0,Rest of the World,36.0611,103.8343,Jan-2020,Gansu,0,0.0
